Tests to run 
- How many total rows are in db and the number of rows we can match with screenshots?
- What is the number of failed hotel checkin combo per execution at
- For the days we have run parquet and db, what is the difference in rows/data in them

In [1]:
# screenshot and parquet file util functions
import re
from pathlib import Path
import pendulum
import pandas as pd
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
base_dir = Path("/Users/nehiljain/code/data/future-dynamics/raw/")
screenshot_filelist = list(base_dir.rglob("*.png"))
print(f"Len of screenshot_filelist: {len(screenshot_filelist)}")
# get all the parquet files that start with `mobile_full_page_screenshot__` or `desktop_full_page_screenshot__` and end with `.parquet`
parquet_filelist = [
    filepath
    for filepath in base_dir.rglob("*.parquet")
    if filepath.name.startswith("mobile_full_page_screenshot__")
    or filepath.name.startswith("desktop_full_page_screenshot__")
]

print(f"Len of parquet_filelist: {len(parquet_filelist)}")

screenshot_filelist_wo_parquet = [
    filepath
    for filepath in screenshot_filelist
    if not any(filepath.stem in parquet_file.stem for parquet_file in parquet_filelist)
]
screenshot_attrs = [
    dict(
        zip(
            ["device", "hotel_name", "checkin_date", "run_at"],
            re.split(r"__", str(filename)),
        )
    )
    for filename in screenshot_filelist
]
screenshot_attrs_df = pd.DataFrame(screenshot_attrs)

screenshot_attrs_df["checkin_date"] = pd.to_datetime(
    screenshot_attrs_df["checkin_date"]
)
screenshot_attrs_df["run_at"] = screenshot_attrs_df["run_at"].str.replace(".png", "")
# use pendulum to parse timestamp to datetime for runat. The format is YYYYMMDDHHMM
screenshot_attrs_df["run_at"] = screenshot_attrs_df["run_at"].apply(
    lambda x: pendulum.from_format(x, "YYYYMMDDHHmm", tz="UTC")
)
screenshot_attrs_df["device"] = screenshot_attrs_df["device"].str.lower()
screenshot_attrs_df["hotel_name"] = screenshot_attrs_df["hotel_name"].str.lower()
print(f"Shape of screenshot_attrs_df: {screenshot_attrs_df.shape}")

# combine all the parquet files into one dataframe, add the filename as a column
parquet_df = pd.concat(
    [
        pd.read_parquet(parquet_file).assign(filename=parquet_file.name)
        for parquet_file in parquet_filelist
    ]
)

parquet_file_attrs = [
    dict(
        zip(
            ["device", "hotel_name", "checkin_date", "run_at"],
            re.split(r"__", str(filename.name).replace(".parquet", "")),
        ),
        filename=str(filename.name),
    )
    for filename in parquet_filelist
]


parquet_file_attrs_df = pd.DataFrame(parquet_file_attrs)
parquet_file_attrs_df["checkin_date"] = pd.to_datetime(
    parquet_file_attrs_df["checkin_date"]
)
parquet_file_attrs_df["device"] = parquet_file_attrs_df["device"].str.replace(
    "_full_page_screenshot", ""
)
parquet_file_attrs_df["run_at"] = parquet_file_attrs_df["run_at"].str.replace(
    ".parquet", ""
)

parquet_file_attrs_df["run_at"] = parquet_file_attrs_df["run_at"].apply(
    lambda x: pendulum.from_format(x, "YYYYMMDDHHmm", tz="UTC")
)
parquet_file_attrs_df["device"] = parquet_file_attrs_df["device"].str.lower()
parquet_file_attrs_df["hotel_name"] = parquet_file_attrs_df["hotel_name"].str.lower()

parquet_file_attrs_df["run_date"] = parquet_file_attrs_df["run_at"].dt.date

print(f"Shape of parquet_df: {parquet_df.shape}")
# filter out rows in parquet_df where `text` col contains `Failed to load`
parquet_df = parquet_df[
    ~parquet_df["text"].str.contains("Failed to load").fillna(False)
]
print(f"Shape of parquet_df after filtering out failed to load: {parquet_df.shape}")
parquet_df = (
    parquet_df.merge(
        parquet_file_attrs_df,
        how="left",
        left_on="filename",
        right_on="filename",
        suffixes=("_df", "_file_attrs"),
    )
    .drop(columns=["run_at_df", "hotel_name_file_attrs"])
    .rename(
        columns={
            "run_at_file_attrs": "run_at",
            "run_date_file_attrs": "run_date",
            "hotel_name_df": "hotel_name",
        }
    )
)
print(
    f"shape of parquet_df after merging with parquet_file_attrs_df: {parquet_df.shape}"
)

Len of screenshot_filelist: 11681
Len of parquet_filelist: 5924
Shape of screenshot_attrs_df: (11681, 4)
Shape of parquet_df: (101530, 11)
Shape of parquet_df after filtering out failed to load: (99396, 11)
shape of parquet_df after merging with parquet_file_attrs_df: (99396, 14)


In [2]:
# FILTER OUT ROWS WHERE `text` COL CONTAINS `Failed to load`  and run_at date is 2024-02-27
parquet_df.groupby("run_date").size()

run_date
2024-01-25     158
2024-01-26      57
2024-01-27     110
2024-01-28      28
2024-01-29     210
2024-01-30      39
2024-01-31     214
2024-02-01     242
2024-02-02     136
2024-02-03     153
2024-02-04     161
2024-02-05      86
2024-02-06     114
2024-02-08      49
2024-02-09      29
2024-02-10     106
2024-02-14     184
2024-02-15      63
2024-02-16      45
2024-02-17      39
2024-02-19     127
2024-02-20      39
2024-02-21     113
2024-02-22      93
2024-02-23    7433
2024-02-24    5819
2024-02-25    6172
2024-02-26    5717
2024-02-27    5848
2024-02-28    6670
2024-02-29    6606
2024-03-01    7126
2024-03-02    7309
2024-03-03    8042
2024-03-04    8017
2024-03-05    7958
2024-03-06    7990
2024-03-07    6094
dtype: int64

In [3]:
# # filter out rows where filename has 20240227 in parquet_df
# # parquet_df[~parquet_df["filename"].str.contains("20240227")].shape
parquet_df[
    ~parquet_df["text"].str.contains("Failed to load").fillna(False)
    & ~parquet_df["text"].isin([None, "None"])
].groupby("run_date").size()

run_date
2024-01-25     158
2024-01-26      57
2024-01-27     110
2024-01-28      28
2024-01-29     210
2024-01-30      39
2024-01-31     214
2024-02-01     242
2024-02-02     136
2024-02-03     153
2024-02-04     161
2024-02-05      86
2024-02-06     114
2024-02-08      49
2024-02-09      29
2024-02-10     106
2024-02-14     184
2024-02-15      63
2024-02-16      45
2024-02-17      39
2024-02-19     127
2024-02-20      39
2024-02-21     113
2024-02-22      93
2024-02-23    7433
2024-02-24    5819
2024-02-25    6172
2024-02-26    5717
2024-02-27    5848
2024-02-28    6670
2024-02-29    6606
2024-03-01    7126
2024-03-02    7309
2024-03-03    8042
2024-03-04    8017
2024-03-05    7958
2024-03-06    7990
2024-03-07    6094
dtype: int64

In [4]:
# # filter parquet_df and screenshot_attrs_df for run_date = "2024-02-27"
# parquet_df = parquet_df[parquet_df["run_at"].dt.date == pd.to_datetime("2024-02-27")]
# screenshot_attrs_df = screenshot_attrs_df[
#     screenshot_attrs_df["run_at"].dt.date == "2024-02-27"
# ]
# print(f"shape of parquet_df after filtering for run_date: {parquet_df.shape}")
# print(
#     f"shape of screenshot_attrs_df after filtering for run_date: {screenshot_attrs_df.shape}"
# )

In [5]:
import duckdb

DUCK_DB_FILE = "/Users/nehiljain/code/data/future-dynamics/raw/hotel_pricer_ai.duckdb"
with duckdb.connect(DUCK_DB_FILE, read_only=True) as con:
    hotel_prices_db = con.execute("SELECT * FROM public.hotel_prices ").df()
    hotel_prices_db["run_at"] = pd.to_datetime(hotel_prices_db["run_at"])
    hotel_prices_db["run_date"] = hotel_prices_db["run_at"].dt.date
    print(f"Shape of hotel_prices_db: {hotel_prices_db.shape}")
    # get total number of unique hotels per run_date from hotel_prices_db
    hotel_prices_db_unique_count = hotel_prices_db.groupby("run_date")[
        "hotel_name"
    ].nunique()
    hotel_prices_db_unique_count = hotel_prices_db_unique_count.reset_index()
    hotel_prices_db_unique_count.columns = ["run_date", "hotel_count"]
    print(hotel_prices_db_unique_count)

Shape of hotel_prices_db: (159375, 8)
      run_date  hotel_count
0   2024-01-23           14
1   2024-01-24           13
2   2024-01-25           13
3   2024-01-26           13
4   2024-01-27           13
5   2024-02-03           14
6   2024-02-06           14
7   2024-02-08           14
8   2024-02-10           14
9   2024-02-15           14
10  2024-02-17           14
11  2024-02-22           14
12  2024-02-23           10
13  2024-02-24           10
14  2024-02-25           11
15  2024-02-26           11
16  2024-02-27           13
17  2024-02-28           12
18  2024-02-29           12
19  2024-03-01           12
20  2024-03-02           12
21  2024-03-03           12
22  2024-03-07           14


In [6]:
# get total number of unique hotels per run_at in screenshot_attrs_df
screenshot_attrs_df["run_at"] = screenshot_attrs_df["run_at"].dt.tz_convert("UTC")
screenshot_attrs_df["run_date"] = screenshot_attrs_df["run_at"].dt.date
# get total number of unique hotels per run_at in screenshot_attrs_df
screenshot_hotel_count = (
    screenshot_attrs_df.groupby("run_date")["hotel_name"].nunique().reset_index()
)

# get total number of unique hotels per run_date in parquet_df
parquet_hotel_count = (
    parquet_df.groupby("run_date")["hotel_name"].nunique().reset_index()
)
# join the hotel_prices_db_unique_count, screenshot_hotel_count and parquet_hotel_count
hotel_count_df = hotel_prices_db_unique_count.merge(
    screenshot_hotel_count, on="run_date", how="outer"
)
hotel_count_df = hotel_count_df.merge(parquet_hotel_count, on="run_date", how="outer")
hotel_count_df.columns = [
    "run_date",
    "hotel_prices_count",
    "screenshot_count",
    "parquet_count",
]
# sort the dataframe by run_date asc
hotel_count_df = hotel_count_df.sort_values("run_date")
print(hotel_count_df)

      run_date  hotel_prices_count  screenshot_count  parquet_count
0   2024-01-23                14.0               NaN            NaN
1   2024-01-24                13.0              11.0            NaN
2   2024-01-25                13.0              12.0            5.0
3   2024-01-26                13.0              11.0            2.0
4   2024-01-27                13.0              11.0            3.0
23  2024-01-28                 NaN              11.0            1.0
24  2024-01-29                 NaN              12.0            5.0
25  2024-01-30                 NaN              12.0            1.0
26  2024-01-31                 NaN              12.0            4.0
27  2024-02-01                 NaN              12.0            5.0
28  2024-02-02                 NaN              12.0            4.0
5   2024-02-03                14.0              12.0            5.0
29  2024-02-04                 NaN              12.0            3.0
30  2024-02-05                 NaN              

In [7]:
import duckdb
import pandas as pd
import pendulum

DUCK_DB_FILE = "/Users/nehiljain/code/data/future-dynamics/raw/hotel_pricer_ai.duckdb"
DATE_FORMAT = "%Y-%m-%d"
DATETIME_FORMAT = "%Y-%m-%d %H:%M:%S"
DATETIME_FORMAT_WITH_MIN = "%Y-%m-%d-%H-%M"
FILTER_DATE = pendulum.datetime(2024, 2, 23, tz="UTC")


def read_hotel_prices_db():
    with duckdb.connect(DUCK_DB_FILE) as con:
        return con.execute("SELECT * FROM public.hotel_prices").df()


def read_price_trends_db():
    with duckdb.connect(DUCK_DB_FILE) as con:
        return con.execute("SELECT * FROM main.price_trends").df()


def read_process_hotel_prices_db():
    with duckdb.connect(DUCK_DB_FILE) as con:
        return con.execute("SELECT * FROM main.hotel_prices").df()


def prepare_parquet_df(parquet_df):
    to_insert_parquet_df = (
        parquet_df.assign(execution_at=lambda x: x.run_at)
        .drop(
            columns=[
                "device_file_attrs",
                "checkin_date_file_attrs",
                "device_df",
                "filename",
                "provider",
                "currency",
                "rate",
            ]
        )
        .rename(
            columns={"hotel_name_df": "hotel_name", "checkin_date_df": "checkin_date"}
        )
    )
    return to_insert_parquet_df


def transform_dates(to_insert_parquet_df):
    to_insert_parquet_df["length_of_stay"] = to_insert_parquet_df[
        "length_of_stay"
    ].astype(int)
    to_insert_parquet_df["run_at"] = pd.to_datetime(
        to_insert_parquet_df["run_at"]
    ).dt.strftime(DATETIME_FORMAT)
    to_insert_parquet_df["run_date"] = pd.to_datetime(
        to_insert_parquet_df["run_date"]
    ).dt.strftime(DATE_FORMAT)
    to_insert_parquet_df["execution_at"] = pd.to_datetime(
        to_insert_parquet_df["execution_at"]
    ).dt.strftime(DATETIME_FORMAT_WITH_MIN)
    return to_insert_parquet_df


def filter_by_run_date(to_insert_parquet_df):
    return to_insert_parquet_df[to_insert_parquet_df["run_at"] >= FILTER_DATE]


def get_column_types(con, table_name):
    query = f"""
    SELECT column_name, data_type 
    FROM information_schema.columns 
    WHERE table_name = '{table_name}' and table_schema='public'
    """
    return con.execute(query).df()


def delsert_into_hotel_prices(to_insert_parquet_df):
    with duckdb.connect(DUCK_DB_FILE) as con:
        con.register("to_insert_parquet_df", to_insert_parquet_df)
        con.execute(
            f"DELETE FROM public.hotel_prices WHERE run_at >= '2024-02-23'::date"
        )
        con.execute(
            """
        INSERT INTO public.hotel_prices 
        (hotel_name, checkin_date, length_of_stay, run_at, run_date, execution_at, scrapped_url, text) 
        SELECT hotel_name, checkin_date, length_of_stay, run_at, run_date, execution_at, scrapped_url, text 
        FROM to_insert_parquet_df
        """
        )


def main(parquet_df):
    to_insert_parquet_df = prepare_parquet_df(parquet_df)
    to_insert_parquet_df = filter_by_run_date(to_insert_parquet_df)
    to_insert_parquet_df = transform_dates(to_insert_parquet_df)

    with duckdb.connect(DUCK_DB_FILE) as con:
        before_num_rows_by_run_date = con.execute(
            "SELECT run_date, COUNT(*) FROM public.hotel_prices GROUP BY run_date order by run_date asc"
        ).df()
        delsert_into_hotel_prices(to_insert_parquet_df)
        after_num_rows_by_run_date = con.execute(
            "SELECT run_date, COUNT(*) FROM public.hotel_prices GROUP BY run_date order by run_date asc"
        ).df()
        # join before and after num_rows_by_run_date on run_date and sort by run_date
        num_rows_by_run_date = before_num_rows_by_run_date.merge(
            after_num_rows_by_run_date, on="run_date", suffixes=("_before", "_after")
        ).sort_values("run_date")
        print(num_rows_by_run_date)


# Assuming `parquet_df` is defined elsewhere in your code
main(parquet_df)

      run_date  count_star()_before  count_star()_after
0   2024-01-23                25906               25906
1   2024-01-24                24120               24120
2   2024-01-25                22086               22086
3   2024-01-26                 2392                2392
4   2024-01-27                 2381                2381
5   2024-02-03                 2821                2821
6   2024-02-06                 2748                2748
7   2024-02-08                 2208                2208
8   2024-02-10                 2557                2557
9   2024-02-15                 1773                1773
10  2024-02-17                 1793                1793
11  2024-02-22                 1753                1753
12  2024-02-23                 7433                7433
13  2024-02-24                 5819                5819
14  2024-02-25                 6172                6172
15  2024-02-26                 5717                5717
16  2024-02-27                 5848             

In [8]:
# filter all our df objects by hotel name Holiday Inn Express & Suites
hotel_filter_condition = (
    lambda x_df: x_df["hotel_name"]
    .str.lower()
    .str.contains("Miami-Biscayne-Bay".lower())
)


def execution_at_filter_condition(x_df):
    df = x_df.copy()
    df["run_date"] = pd.to_datetime(df["run_date"])
    return df["run_date"] == pd.to_datetime("2024-02-27")


raw_hotel_prices_db = read_hotel_prices_db()
raw_hotel_prices_db_filtered = raw_hotel_prices_db[
    execution_at_filter_condition(raw_hotel_prices_db)
]
screenshot_attrs_df_filtered = screenshot_attrs_df[
    execution_at_filter_condition(screenshot_attrs_df)
]
parquet_df_filtered = parquet_df[execution_at_filter_condition(parquet_df)]

screenshot_hotel_filter_condition = screenshot_attrs_df_filtered[
    "hotel_name"
].str.contains("Miami-Biscayne-Bay")

# count number of rows per run date in parquet_df filtered by hotel_filter_condition
print(
    parquet_df_filtered[hotel_filter_condition(parquet_df_filtered)]
    .groupby("run_date")
    .size()
)

# count number of rows per run date in hotel prices filtered by hotel_filter_condition
print(
    raw_hotel_prices_db_filtered[hotel_filter_condition(raw_hotel_prices_db_filtered)]
    .groupby("run_date")
    .size()
)

Series([], dtype: int64)
Series([], dtype: int64)


In [9]:
# count number of rows per run date in price trends filtered by hotel_filter_condition
price_trends_db = read_price_trends_db()
price_trends_db.head()
hotel_filter_condition(price_trends_db)


def read_dbt_hotel_prices_db():
    with duckdb.connect(DUCK_DB_FILE) as con:
        return con.execute("SELECT * FROM main.hotel_prices").df()


dbt_hotel_prices_df = read_dbt_hotel_prices_db()
dbt_hotel_prices_df_filtered = dbt_hotel_prices_df[
    execution_at_filter_condition(dbt_hotel_prices_df)
]
# count number of rows per run date in dbt_hotel_prices_df filtered by hotel_filter_condition
print(
    dbt_hotel_prices_df_filtered[hotel_filter_condition(dbt_hotel_prices_df_filtered)]
    .groupby("run_date")
    .size()
)

# # check if hotel_filter_condition(dbt_hotel_prices_df) is the same as hotel_filter_condition(hotel_prices_db)
# assert hotel_filter_condition(dbt_hotel_prices_df).equals(
#     hotel_filter_condition(hotel_prices_db)
# )

# print all rows in dbt_hotel_prices_df is different from hotel_prices_db after filtering by hotel_filter_condition
# filtered_dbt_hotel_prices_df = dbt_hotel_prices_df[
#     hotel_filter_condition(dbt_hotel_prices_df)
# ]
# filtered_hotel_prices_db = hotel_prices_db[hotel_filter_condition(hotel_prices_db)]

# diff_rows = filtered_dbt_hotel_prices_df[
#     ~filtered_dbt_hotel_prices_df.isin(filtered_hotel_prices_db)
# ].dropna()
# print(diff_rows)

Series([], dtype: int64)


In [10]:
import pandas as pd

rawish_hotel_prices_db = pd.read_parquet(
    "/Users/nehiljain/code/future-dynamics-hotels/future_dynamics_bi/rawish_hotel_prices_df.parquet"
)
rawish_hotel_prices_db["run_date"] = pd.to_datetime(rawish_hotel_prices_db["run_date"])
rawish_hotel_prices_db_filtered = rawish_hotel_prices_db[
    execution_at_filter_condition(rawish_hotel_prices_db)
]
# count number of rows per run date in rawish_hotel_prices_db filtered
# rawish_hotel_prices_db_filtered = rawish_hotel_prices_db_filtered[
#     hotel_filter_condition(rawish_hotel_prices_db_filtered)
# ]
rawish_hotel_prices_db_filtered = rawish_hotel_prices_db_filtered[
    rawish_hotel_prices_db_filtered["run_date"] == pd.to_datetime("2024-02-27")
]
rawish_hotel_prices_db_filtered["ai_input"].values

array(['Chelsea Hotel Toronto__2024-04-10__1__2024-02-27->None',
       'Chelsea Hotel Toronto__2024-04-10__1__2024-02-27->None',
       'Chelsea Hotel Toronto__2024-04-10__1__2024-02-27->None', ...,
       'Hotel Hayden__2024-03-30__1__2024-02-27->None',
       'Hotel Hayden__2024-03-30__1__2024-02-27->None',
       'Hotel Hayden__2024-03-30__1__2024-02-27->None'], dtype=object)

In [11]:
price_trends_db = read_price_trends_db()
price_trends_db.head()
# get num rows in price_trends_db filtered by hotel_filter_condition and exec_Date >= 2024-02-23 by exec_Date
price_trends_db.groupby("exec_date").size()
# # price_trends_db_filtered = price_trends_db[hotel_filter_condition(price_trends_db)]
# price_trends_db_filtered = price_trends_db[
#     price_trends_db["exec_date"] == pd.to_datetime("2024-02-21")
# ]
# price_trends_db_filtered["ai_input"].values

exec_date
2024-01-24 00:00:00-05:00    526
2024-01-25 00:00:00-05:00    459
2024-01-26 00:00:00-05:00     51
2024-02-03 00:00:00-05:00     56
2024-02-06 00:00:00-05:00    103
2024-02-08 00:00:00-05:00     48
2024-02-14 00:00:00-05:00     32
2024-02-17 00:00:00-05:00     35
2024-02-21 00:00:00-05:00     36
2024-02-22 00:00:00-05:00     30
2024-02-23 00:00:00-05:00    169
2024-02-24 00:00:00-05:00    129
2024-02-25 00:00:00-05:00    139
2024-02-26 00:00:00-05:00    133
2024-02-27 00:00:00-05:00    163
2024-02-28 00:00:00-05:00    162
2024-02-29 00:00:00-05:00    158
2024-03-01 00:00:00-05:00    151
2024-03-02 00:00:00-05:00    166
2024-03-03 00:00:00-05:00    122
dtype: int64

In [12]:
def read_hotel_prices_db():
    with duckdb.connect(DUCK_DB_FILE) as con:
        return con.execute("SELECT * FROM public.hotel_prices").df()


def read_price_trends_db():
    with duckdb.connect(DUCK_DB_FILE) as con:
        return con.execute("SELECT * FROM main.price_trends").df()


def read_process_hotel_prices_db():
    with duckdb.connect(DUCK_DB_FILE) as con:
        return con.execute("SELECT * FROM main.hotel_prices").df()


raw_hotel_price_df = read_hotel_prices_db()
processed_hotel_price_df = read_process_hotel_prices_db()
price_trends_df = read_price_trends_db()

In [13]:
print(raw_hotel_price_df.groupby("run_date").size())
print(processed_hotel_price_df.groupby("run_date").size())
print(price_trends_df.groupby("exec_date").size())

run_date
2024-01-23    25906
2024-01-24    24120
2024-01-25    22086
2024-01-26     2392
2024-01-27     2381
2024-02-03     2821
2024-02-06     2748
2024-02-08     2208
2024-02-10     2557
2024-02-15     1773
2024-02-17     1793
2024-02-22     1753
2024-02-23     7433
2024-02-24     5819
2024-02-25     6172
2024-02-26     5717
2024-02-27     5848
2024-02-28     6670
2024-02-29     6606
2024-03-01     7126
2024-03-02     7309
2024-03-03     8042
2024-03-04     8017
2024-03-05     7958
2024-03-06     7990
2024-03-07     6094
dtype: int64
run_date
2024-01-23    21725
2024-01-24    18516
2024-01-25    16485
2024-01-26     1799
2024-01-27     1831
2024-02-03     2051
2024-02-06     2084
2024-02-08     1582
2024-02-10     1759
2024-02-15     1030
2024-02-17      981
2024-02-22     1016
2024-02-23     4594
2024-02-24     3656
2024-02-25     4020
2024-02-26     3733
2024-02-27     3847
2024-02-28     4181
2024-02-29     4235
2024-03-01     4183
2024-03-02     4108
2024-03-03     4484
dtype: in

In [14]:
# import re

# import pandas as pd
# import pendulum


# def create_regex_patterns(hotel_name):
#     """Create regex patterns based on the hotel name"""
#     relevant_row_regex = re.compile(
#         rf"{re.escape(hotel_name)}.*?\$\d+.*?Visit site", re.IGNORECASE
#     )
#     irrelevant_row_regex = re.compile(
#         rf"^(?!.*{re.escape(hotel_name)}.*?\$\d+.*?Visit site).*", re.IGNORECASE
#     )
#     provider_price_row_regex = re.compile(
#         rf"{re.escape(hotel_name)}.*?->(.*?);;\$(\d+)", re.IGNORECASE
#     )
#     return relevant_row_regex, irrelevant_row_regex, provider_price_row_regex


# def is_relevant_row(hotel_name, text):
#     """Check if the row is relevant to the hotel name"""
#     relevant_row_regex, _, _ = create_regex_patterns(hotel_name)
#     return bool(relevant_row_regex.search(text))


# def get_list_price_usd(hotel_name, text):
#     """Get the list price in USD from the text"""
#     _, _, provider_price_row_regex = create_regex_patterns(hotel_name)
#     return float(provider_price_row_regex.search(text).group(2))


# def get_list_price_provider(hotel_name, text):
#     """Get the list price provider from the text"""
#     _, _, provider_price_row_regex = create_regex_patterns(hotel_name)
#     return provider_price_row_regex.search(text).group(1)


# def parse_date_string(date_string):
#     return pendulum.parse(date_string, strict=False).set(tz="UTC")


# def model(raw_df):
#     print(f"shape of raw df: {raw_df.shape}")
#     print(f"cols of raw df: {raw_df.columns}")
#     df = raw_df.copy()
#     print(f"shape of df: {df.shape}")
#     df["run_date"] = pd.to_datetime(df["run_date"])
#     df["run_at"] = pd.to_datetime(df["run_at"])
#     # filter df for date 2024-02-27
#     df = df[df["run_date"] == "2024-02-27"]
#     print(f"shape of df after filtering for run_date: {df.shape}")

#     df["ai_input"] = df.apply(
#         lambda x: f"{x['hotel_name']}__{x['checkin_date']}__{x['length_of_stay']}__{x['run_date']}->{x['text']}",
#         axis=1,
#     )
#     print(f"unique text: {df['text'].unique()}")
#     print(f"unique ai_input: {df['ai_input'].unique()}")
#     relevant_row_regex, _, provider_price_row_regex = create_regex_patterns(
#         df["hotel_name"].values[0]
#     )
#     df["relevant"] = df.apply(
#         lambda x: is_relevant_row(x["hotel_name"], x["ai_input"]), axis=1
#     )
#     df = df[df["relevant"]]
#     print(f"shape of df after filtering for revelvant: {df.shape}")
#     df["list_price_usd"] = df.apply(
#         lambda x: get_list_price_usd(x["hotel_name"], x["ai_input"]),
#         axis=1,
#     )

#     df["list_price_provider"] = df.apply(
#         lambda x: get_list_price_provider(x["hotel_name"], x["ai_input"]), axis=1
#     )
#     # add a column for device type mobile or desktop based on the searching mobile or deskptop in the text
#     df["device_type"] = df["text"].apply(
#         lambda x: "mobile" if "mobile" in x else "desktop"
#     )
#     df["run_at"] = pd.to_datetime(df["run_at"])
#     df["run_date"] = pd.to_datetime(df["run_date"])
#     df["execution_at_dt"] = pd.to_datetime(df["execution_at"].apply(parse_date_string))
#     df["execution_date"] = df["execution_at_dt"].dt.date
#     # drop ai_input, relevant col
#     df = df.drop(columns=["ai_input", "relevant"])
#     print(f"shape of df before final return: {df.shape}")
#     return df


# processed_hotel_price_df = model(raw_hotel_price_df)

In [15]:
# raw_hotel_price_df_filtered = where text doesnt have failed to load
raw_hotel_price_df_filtered = raw_hotel_price_df[
    ~raw_hotel_price_df["text"].str.contains("Failed to load").fillna(False)
]
# processed_hotel_price_df = where text doesnt have failed to load
processed_hotel_price_df_filtered = processed_hotel_price_df[
    ~processed_hotel_price_df["text"].str.contains("Failed to load").fillna(False)
]
# filter for date 2024-02-27
raw_hotel_price_df_filtered = raw_hotel_price_df_filtered[
    raw_hotel_price_df_filtered["run_date"] == "2024-02-27"
]
# filter values from raw_hotel_price_df_filtered where text is None or "None"
raw_hotel_price_df_filtered = raw_hotel_price_df_filtered[
    ~raw_hotel_price_df_filtered["text"].isin([None, "None"])
]
# processed_hotel_price_df_filtered = processed_hotel_price_df_filtered[
#     processed_hotel_price_df_filtered["run_date"] == "2024-02-27"
# ]
# print shapes
print(raw_hotel_price_df_filtered.shape)
print(processed_hotel_price_df_filtered.shape)

(5848, 8)
(111900, 13)


In [16]:
# get row count for all hotel_name and checkin date combo in raw_hotel_price_df_filtered
raw_hotel_price_df.groupby("run_date").size()

run_date
2024-01-23    25906
2024-01-24    24120
2024-01-25    22086
2024-01-26     2392
2024-01-27     2381
2024-02-03     2821
2024-02-06     2748
2024-02-08     2208
2024-02-10     2557
2024-02-15     1773
2024-02-17     1793
2024-02-22     1753
2024-02-23     7433
2024-02-24     5819
2024-02-25     6172
2024-02-26     5717
2024-02-27     5848
2024-02-28     6670
2024-02-29     6606
2024-03-01     7126
2024-03-02     7309
2024-03-03     8042
2024-03-04     8017
2024-03-05     7958
2024-03-06     7990
2024-03-07     6094
dtype: int64

In [17]:
processed_hotel_price_df.groupby("run_date").size()

run_date
2024-01-23    21725
2024-01-24    18516
2024-01-25    16485
2024-01-26     1799
2024-01-27     1831
2024-02-03     2051
2024-02-06     2084
2024-02-08     1582
2024-02-10     1759
2024-02-15     1030
2024-02-17      981
2024-02-22     1016
2024-02-23     4594
2024-02-24     3656
2024-02-25     4020
2024-02-26     3733
2024-02-27     3847
2024-02-28     4181
2024-02-29     4235
2024-03-01     4183
2024-03-02     4108
2024-03-03     4484
dtype: int64